---
Check that we don't have python 3.10.2 (otherwise PyTorch won't work)

In [10]:
import sys
print(sys.version)

3.9.10 (tags/v3.9.10:f2f3f53, Jan 17 2022, 15:14:21) [MSC v.1929 64 bit (AMD64)]


---
Imports - most importantly clubs_gym (https://github.com/fschlatt/clubs_gym)

In [11]:
import gym
import clubs_gym
import time
import random
import numpy as np

---
List of available environments

In [12]:
clubs_gym.ENVS

['LeducTwoPlayer-v0',
 'KuhnTwoPlayer-v0',
 'KuhnThreePlayer-v0',
 'LimitHoldemTwoPlayer-v0',
 'LimitHoldemSixPlayer-v0',
 'LimitHoldemNinePlayer-v0',
 'NoLimitHoldemTwoPlayer-v0',
 'NoLimitHoldemSixPlayer-v0',
 'NoLimitHoldemNinePlayer-v0',
 'NoLimitHoldemBbAnteNinePlayer-v0',
 'PotLimitOmahaTwoPlayer-v0',
 'PotLimitOmahaSixPlayer-v0',
 'PotLimitOmahaNinePlayer-v0',
 'ShortDeckTwoPlayer-v0',
 'ShortDeckSixPlayer-v0',
 'ShortDeckNinePlayer-v0']

We will only focus on `NoLimitHoldemTwoPlayer-v0` and `NoLimitHoldemSixPlayer-v0`

---
Let's open `NoLimitHoldemTwoPlayer-v0`.

In [13]:
env_str = "NoLimitHoldemTwoPlayer-v0"
num_players = 2  # heads-up
num_streets = 4  # texas hold'em
blinds = [1, 2]  
antes = 0  # a bit weird to have antes in a heads-up I reckon (single ints are expanded to the number of players so could've inputted a list)
raise_sizes = ['inf', 'inf', 'inf', 'inf']  # yes the documentation is wrong, 'inf' should be used and not float('inf')
num_raises = 'inf'  # max number of bets for each street* (including preflop)
num_suits = 4  # number of suits to use in deck (obviously 4 otherwise I mean it's weird)
num_ranks = 13  # same, obviously 13 (A, 2, 3, 4, 5, 6, 7, 8, 9, T, J, Q, K)
num_hole_cards = 2  # **
mandatory_num_hole_cards = 0  # you can use the fives on the board and it will end up being a draw if your enemy does it too
num_community_cards = [0, 3, 1, 1]  # preflop, flop, turn, river
start_stack = 100 
n_cards_for_hand = 5  # texas hold'em
order = ['sf', 'fk', 'fh', 'fl', 'st', 'tk', 'tp', 'pa', 'hc']  # most likely optional, but just to make sure we're working with the right order

config_dict = {'num_players': num_players,
               'num_streets': num_streets,
               'blinds': blinds,
               'antes': antes,
               'raise_sizes': raise_sizes,
               'num_raises': num_raises,
               'num_suits': num_suits,
               'num_ranks': num_ranks,
               'num_hole_cards': num_hole_cards,
               'mandatory_num_hole_cards': mandatory_num_hole_cards,
               'num_community_cards': num_community_cards,
               'start_stack': start_stack,
               'num_cards_for_hand': n_cards_for_hand,
               'order': order}


---
Some vocabulary:
- \* A card that’s dealt in a particular betting round, in many poker variants, is called a **street**. For example, in Texas Hold’em and Omaha, the community cards are referred to as the flop, turn and river, with the turn and river sometimes referred to as Fourth Street and Fifth Street.
- ** The term **hole cards' in poker signals that the cards are private cards that should only be viewed by the player. The dealer deals hole cards face down and players should ensure that the value of those cards remains secret.

---
`obs` is really hard to read, thus we created a `print_obs` function to get nice prints on what is happening.

In [14]:
def print_obs(obs):
    """
    Fancy print of stats for the clubs gym poker gym.
    All Cards displayed (we see everybody's cards)
    
    param: obs: first argument returned by env.step
    return: none
    """
    for _ in range(5):
        print('---')
    print('------------------- Players\' stacks -------------------')
    print(f"player_0: {obs['stacks'][0]}\n" + \
          f"player_1: {obs['stacks'][1]}")
    print('------------------- Game Stats -------------------')
    print(f'community_cards: {obs["community_cards"]}')
    print(f'Dealer/Button: {obs["button"] % 2}')
    print(f'Pot: {obs["pot"]}')
    print(f'Commits: player_0: {obs["street_commits"][0]} |||  player_1: {obs["street_commits"][1]}')

    if not obs['action'] == -1:
        print(f"\n---------------- player {obs['action']}'s turn ----------------")
        print(f'hole_cards: {obs["hole_cards"]}')
        print(f'call: {obs["call"]}, min_raise: {obs["min_raise"]}, max_raise: {obs["max_raise"]}', end='\n\n')
    else:
        print('\n[INFO] - End of the Game.')

---
# Main

`env.act` handles the agents turns add calls `act` method of agent. All agent require an `act()` method.

`env.step` asks for an integer corresponding to the amount you want to bet. 
- If the bet is less than the minimum bet, you fold. 
- If the bet is higher than the maximum bid (which in Texas Hold'em not limit means that you want to bet more than you have) then you go all-in.

Here, we code three agents : AllInFoldQLearningAgent, RandomAllInFold and HumanRandomAllInFold.

In [15]:
from clubs_gym.agent.base import BaseAgent
class RandomAgent(BaseAgent):
    def __init__(self, player_id, seed = 42):
        self.rand_generator = np.random.RandomState(seed)
        self.player_id = player_id

    def act(self, obs):

        available_chips = obs['stacks'][self.player_id]
        action = self.rand_generator.randint(available_chips)
        return(action)

In [16]:
from clubs_gym.agent.base import BaseAgent
class HumanAgent(BaseAgent):
    def __init__(self, player_id, seed = 42):
        self.rand_generator = np.random.RandomState(seed)
        self.player_id = player_id

    def act(self, obs):
        available_chips = obs['stacks'][self.player_id]
        action = -1
        while action > available_chips or action < 0:
            action = int(input(f"C'est à {self.player_id} de jouer"))
        return(action)

In [ ]:

clubs_gym.envs.register({env_str: config_dict})
env = gym.make(env_str)
env.register_agents([HumanAgent(0),RandomAgent(1)])

# Start the game
obs = env.reset(reset_stacks = True, reset_button = True)
print(f"Action space is of size: {env.action_space}")

print("---\nGame started\n---")
counter = 0

# print(f"Hand number {counter + 1}\n---")

print("A")
print_obs(obs)
num_episodes = 1000
for _ in range(num_episodes):
    while True:
        
        bet = env.act(obs)
        obs, rewards, done, info = env.step(bet)  
        print(f"ID of the action that was taken : {bet}")
        print("AA")
        print_obs(obs)
        if all(done):
            print(rewards)
            print(obs["active"])
            env.reset()
            print("AAA")
            print_obs(obs)
            break

env.close()

Action space is of size: Discrete(200)
---
Game started
---
A
---
---
---
---
---
------------------- Players' stacks -------------------
player_0: 99
player_1: 98
------------------- Game Stats -------------------
community_cards: []
Dealer/Button: 0
Pot: 3
Commits: player_0: 1 |||  player_1: 2

---------------- player 0's turn ----------------
hole_cards: [Card (2501257322848): 6♥, Card (2501257759088): A♣]
call: 1, min_raise: 3, max_raise: 99



C'est à 0 de jouer 0


ID of the action that was taken : 0
AA
---
---
---
---
---
------------------- Players' stacks -------------------
player_0: 99
player_1: 101
------------------- Game Stats -------------------
community_cards: []
Dealer/Button: 0
Pot: 0
Commits: player_0: 1 |||  player_1: 2

[INFO] - End of the Game.
[-1, 1]
[False, True]
AAA
---
---
---
---
---
------------------- Players' stacks -------------------
player_0: 99
player_1: 101
------------------- Game Stats -------------------
community_cards: []
Dealer/Button: 0
Pot: 0
Commits: player_0: 1 |||  player_1: 2

[INFO] - End of the Game.
ID of the action that was taken : 51
AA
---
---
---
---
---
------------------- Players' stacks -------------------
player_0: 97
player_1: 49
------------------- Game Stats -------------------
community_cards: []
Dealer/Button: 1
Pot: 54
Commits: player_0: 2 |||  player_1: 52

---------------- player 0's turn ----------------
hole_cards: [Card (2501257323184): 8♣, Card (2501257324528): 5♥]
call: 50, min_ra

C'est à 0 de jouer 0


ID of the action that was taken : 0
AA
---
---
---
---
---
------------------- Players' stacks -------------------
player_0: 97
player_1: 103
------------------- Game Stats -------------------
community_cards: [Card (2501257323664): 6♣, Card (2501260518400): 3♣, Card (2501257324720): T♠, Card (2501257281104): T♣, Card (2501257280768): J♦]
Dealer/Button: 1
Pot: 0
Commits: player_0: 0 |||  player_1: 0

[INFO] - End of the Game.
[-2, 2]
[False, True]
AAA
---
---
---
---
---
------------------- Players' stacks -------------------
player_0: 97
player_1: 103
------------------- Game Stats -------------------
community_cards: [Card (2501257323664): 6♣, Card (2501260518400): 3♣, Card (2501257324720): T♠, Card (2501257281104): T♣, Card (2501257280768): J♦]
Dealer/Button: 1
Pot: 0
Commits: player_0: 0 |||  player_1: 0

[INFO] - End of the Game.


...

ID of the action that was taken : 0
AA
---
---
---
---
---
------------------- Players' stacks -------------------
player_0: 91
player_1: 109
------------------- Game Stats -------------------
community_cards: [Card (2501257324096): 8♥, Card (2501257281344): Q♠, Card (2501260436192): 2♠, Card (2501257323808): 4♥, Card (2501257323856): 7♠]
Dealer/Button: 1
Pot: 0
Commits: player_0: 0 |||  player_1: 0

[INFO] - End of the Game.
[-2, 2]
[False, True]
AAA
---
---
---
---
---
------------------- Players' stacks -------------------
player_0: 91
player_1: 109
------------------- Game Stats -------------------
community_cards: [Card (2501257324096): 8♥, Card (2501257281344): Q♠, Card (2501260436192): 2♠, Card (2501257323808): 4♥, Card (2501257323856): 7♠]
Dealer/Button: 1
Pot: 0
Commits: player_0: 0 |||  player_1: 0

[INFO] - End of the Game.


C'est à 0 de jouer 0


ID of the action that was taken : 0
AA
---
---
---
---
---
------------------- Players' stacks -------------------
player_0: 90
player_1: 110
------------------- Game Stats -------------------
community_cards: []
Dealer/Button: 0
Pot: 0
Commits: player_0: 1 |||  player_1: 2

[INFO] - End of the Game.
[-1, 1]
[False, True]
AAA
---
---
---
---
---
------------------- Players' stacks -------------------
player_0: 90
player_1: 110
------------------- Game Stats -------------------
community_cards: []
Dealer/Button: 0
Pot: 0
Commits: player_0: 1 |||  player_1: 2

[INFO] - End of the Game.
ID of the action that was taken : 106
AA
---
---
---
---
---
------------------- Players' stacks -------------------
player_0: 88
player_1: 3
------------------- Game Stats -------------------
community_cards: []
Dealer/Button: 1
Pot: 109
Commits: player_0: 2 |||  player_1: 107

---------------- player 0's turn ----------------
hole_cards: [Card (2501257323808): 4♥, Card (2501257280768): J♦]
call: 88, min_

C'est à 0 de jouer 0


ID of the action that was taken : 0
AA
---
---
---
---
---
------------------- Players' stacks -------------------
player_0: 88
player_1: 112
------------------- Game Stats -------------------
community_cards: [Card (2501257324576): 4♦, Card (2501257281344): Q♠, Card (2501257323136): 7♣, Card (2501257044272): Q♣, Card (2501257324480): 9♥]
Dealer/Button: 1
Pot: 0
Commits: player_0: 0 |||  player_1: 0

[INFO] - End of the Game.
[-2, 2]
[False, True]
AAA
---
---
---
---
---
------------------- Players' stacks -------------------
player_0: 88
player_1: 112
------------------- Game Stats -------------------
community_cards: [Card (2501257324576): 4♦, Card (2501257281344): Q♠, Card (2501257323136): 7♣, Card (2501257044272): Q♣, Card (2501257324480): 9♥]
Dealer/Button: 1
Pot: 0
Commits: player_0: 0 |||  player_1: 0

[INFO] - End of the Game.


In [ ]:
import gym

import clubs_gym

env = gym.make("KuhnTwoPlayer-v0")
env.register_agents([clubs_gym.agent.kuhn.NashKuhnAgent(0.3)] * 2)
obs = env.reset()

while True:
    bet = env.act(obs)
    obs, rewards, done, info = env.step(bet)
    print(obs)

    if all(done):
        break

print(rewards)